<a href="https://colab.research.google.com/github/wiamfa/Speech_emotion_recognition/blob/main/huggingFace_Wav2vec-FT-Ravdess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate>=0.20.1
!accelerate config
!accelerate env


In [2]:
model_checkpoint = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
batch_size = 8
ravdess_emotions = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]



In [ ]:
!pip install datasets
!pip install transformers
!pip install librosa

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
%%capture
!apt install git-lfs

In [7]:
from transformers.utils import send_example_telemetry

send_example_telemetry("audio_classification_notebook", framework="pytorch")

#test HuggingFace on Hubert

In [8]:
from datasets import load_dataset, load_metric, Audio

In [9]:
dataset = load_dataset("xbgoose/ravdess",split="train")
metric = load_metric("accuracy")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1440 [00:00<?, ? examples/s]

<ipython-input-9-0b02b7a0bda2>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [10]:
dataset000=dataset.remove_columns([ 'modality', 'vocal_channel', 'emotional_intensity', 'statement', 'repetition', 'actor', 'gender'])

In [11]:
from datasets.tasks.text_classification import ClassLabel
#resampling to 16Khz
dataset00 = dataset000.cast_column("audio", Audio(sampling_rate=16000))
#Label identification
dataset0 = dataset00.cast_column("emotion", ClassLabel(num_classes=8, names=ravdess_emotions))


Casting the dataset:   0%|          | 0/1440 [00:00<?, ? examples/s]

In [42]:
dataset1=dataset0.train_test_split(test_size=0.1)

In [44]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 1296
    })
    test: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 144
    })
})

In [45]:
dataset1["test"][100]

{'audio': {'path': '03-01-02-01-01-01-03.wav',
  'array': array([ 1.55431223e-15, -2.13162821e-14, -7.06101844e-14, ...,
         -5.97731414e-05, -7.34426139e-05, -3.65047235e-05]),
  'sampling_rate': 16000},
 'emotion': 1}

In [46]:
dataset1["train"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'emotion': ClassLabel(names=['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised'], id=None)}

In [47]:
labels = dataset1["train"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["7"]

'surprised'

In [48]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset1["train"])-1)
    example = dataset1["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["emotion"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: happy
Shape: (67268,), sampling rate: 16000



Label: happy
Shape: (57124,), sampling rate: 16000



Label: angry
Shape: (58192,), sampling rate: 16000



Label: sad
Shape: (67268,), sampling rate: 16000



Label: calm
Shape: (59794,), sampling rate: 16000


In [49]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [50]:
max_duration = 2.0  # seconds

In [51]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [52]:
preprocess_function(dataset1['train'][:5])

{'input_values': [array([ 5.3529348e-04,  9.7076857e-04,  4.2026828e-04, ...,
       -1.5969950e-01,  3.3404207e-01,  7.8168786e-01], dtype=float32), array([ 3.6376021e-03,  3.4015509e-03,  4.6211356e-04, ...,
       -4.0638466e+00, -4.1790833e+00, -2.9688418e+00], dtype=float32), array([4.2200994e-04, 4.2200994e-04, 4.2200994e-04, ..., 6.8690062e-01,
       7.6025558e-01, 7.6485467e-01], dtype=float32), array([ 7.3558789e-05,  7.3558789e-05,  7.3558789e-05, ...,
       -2.0029848e+00, -1.4385210e+00, -1.3497221e+00], dtype=float32), array([-1.3180650e-03, -8.1519975e-04, -2.0544075e-03, ...,
        1.8446801e+00,  1.2822294e+00,  6.1094612e-01], dtype=float32)], 'attention_mask': [array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]}

In [53]:
encoded_dataset = dataset1.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

Map:   0%|          | 0/1296 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['emotion', 'input_values', 'attention_mask'],
        num_rows: 1296
    })
    test: Dataset({
        features: ['emotion', 'input_values', 'attention_mask'],
        num_rows: 144
    })
})

In [54]:
encoded_dataset["train"][0]["emotion"]

2

In [55]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['projector.weight', 'classifier.weight', 'projector.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ravdess-v7",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    gradient_checkpointing=True,


)

In [57]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [58]:
encoded_dataset["train"]

Dataset({
    features: ['emotion', 'input_values', 'attention_mask'],
    num_rows: 1296
})

In [59]:
encoded_dataset1= encoded_dataset.rename_column("emotion", "labels")

In [67]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset1["train"],
    eval_dataset=encoded_dataset1["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [72]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
25,0.251800,1.081302,0.722222
50,0.437700,1.367777,0.638889
75,0.471000,1.284124,0.645833
100,0.690600,1.084484,0.673611
125,0.840900,0.998660,0.722222
150,0.591200,0.902895,0.729167
175,0.602900,1.086249,0.659722
200,0.452500,1.045476,0.680556
225,0.426300,1.420858,0.638889
250,0.486600,1.064764,0.722222


TrainOutput(global_step=810, training_loss=0.316248012729633, metrics={'train_runtime': 1221.4988, 'train_samples_per_second': 5.305, 'train_steps_per_second': 0.663, 'total_flos': 3.9278526345216e+17, 'train_loss': 0.316248012729633, 'epoch': 5.0})

In [73]:
trainer.evaluate()

{'eval_loss': 0.8319895267486572,
 'eval_accuracy': 0.7986111111111112,
 'eval_runtime': 7.658,
 'eval_samples_per_second': 18.804,
 'eval_steps_per_second': 4.701,
 'epoch': 5.0}

In [74]:
trainer.push_to_hub()

'https://huggingface.co/Wiam/wav2vec2-large-xlsr-53-english-finetuned-ravdess-v7/tree/main/'

In [65]:
import torch
torch.cuda.empty_cache()